In [17]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Load the dataset
parquet_file =  r'B:\Python\Veridion Project\Project 4\veridion_product_deduplication_challenge.snappy.parquet'
df = pd.read_parquet(parquet_file, engine="auto")
print (df)


                                                  unspsc  \
0      Sewing and stitchery and weaving equipment and...   
1                 Electric alternating current AC motors   
2                     Vehicle trim and exterior covering   
3                                        Pipe connectors   
4                                                  Doors   
...                                                  ...   
21941                                              Other   
21942                     Processed and synthetic rubber   
21943                            Fresh cut rose bouquets   
21944  Vision correction or cosmetic eyewear and rela...   
21945       Chocolate and sugars and sweetening products   

                         root_domain  \
0                  studio-atcoat.com   
1                     worm-gears.net   
2               customcarcoverco.com   
3                    plumbmaster.com   
4                           sogno.in   
...                              ...   

In [ ]:
empty_products = df[df['description'].isnull() | df['description'].str.strip().eq('') | df['product_summary'].isnull() | df['product_summary'].str.strip().eq('')]

# Display products with empty descriptions or summaries
# print(empty_products[['unspsc', 'product_identifier', 'description', 'product_summary']])
df_clean = df[~df['product_identifier'].isin(empty_products['product_identifier'])]
grouped = df_clean.groupby('unspsc')

print(grouped)

In [8]:

#check similarity within each group and remove duplicates
def remove_duplicates_within_group(group):
    # Concatenate 'description' and 'product_summary' to create a single text column
    group['combined_text'] = group['description'].fillna('') + " " + group['product_summary'].fillna('')
    group = group[group['combined_text'].str.strip() != '']
    
    if group.empty:
        return group  # Return empty group as-is (no products to process)
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(group['combined_text'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    to_keep = set()
    for i in range(len(group)):
        if i not in to_keep:
            to_keep.add(i) 
            for j in range(i + 1, len(group)):
                if cosine_sim[i, j] > 0.8:
                    to_keep.add(j)
    return group.iloc[list(to_keep)]


In [10]:

cleaned_df = pd.concat([remove_duplicates_within_group(group) for _, group in tqdm(grouped, desc="Removing duplicates")])
print("Cleaned DataFrame with duplicates removed and empty descriptions/summaries eliminated:")
print(cleaned_df[['unspsc', 'product_identifier', 'description', 'product_summary']])


Removing duplicates: 100%|████████████████████████████████████████████████████████| 1492/1492 [00:10<00:00, 147.12it/s]


Cleaned DataFrame with duplicates removed and empty descriptions/summaries eliminated:
                unspsc product_identifier  \
741    Abrasive wheels                 []   
6649   Abrasive wheels                 []   
6845   Abrasive wheels                 []   
7742   Abrasive wheels                 []   
9137   Abrasive wheels                 []   
...                ...                ...   
21381            Yarns                 []   
21529            Yarns                 []   
21634            Yarns                 []   
21707            Yarns                 []   
21872            Yarns                 []   

                                             description  \
741    "Felt Wheels" are available in a variety of sh...   
6649   The "Abrasive Grinding Wheel" manufactured by ...   
6845   The "Diamond Cutting Disc Blade" manufactured ...   
7742   The "Diamond Turbo Blade Cutting Disc Wheel" i...   
9137   The "Abrasive Grinding Wheel" produced by Huaz...   
...         

In [12]:
cleaned_df = pd.concat([remove_duplicates_within_group(group) for _, group in tqdm(grouped, desc="Removing duplicates")])
print("Cleaned DataFrame with duplicates removed and empty descriptions/summaries eliminated:")
print(cleaned_df)


Removing duplicates: 100%|████████████████████████████████████████████████████████| 1492/1492 [00:10<00:00, 145.35it/s]


Cleaned DataFrame with duplicates removed and empty descriptions/summaries eliminated:
                unspsc               root_domain  \
741    Abrasive wheels                 refima.eu   
6649   Abrasive wheels          diamond-tool.net   
6845   Abrasive wheels          diamond-tool.net   
7742   Abrasive wheels          diamond-tool.net   
9137   Abrasive wheels          diamond-tool.net   
...                ...                       ...   
21381            Yarns       sallyridgway.com.au   
21529            Yarns  meanmotherscreations.com   
21634            Yarns      yarnswithalix.com.au   
21707            Yarns    edencottageyarns.co.uk   
21872            Yarns     wattlebirdyarn.com.au   

                                                page_url  \
741             https://www.refima.eu/technical-support/   
6649   http://www.diamond-tool.net/product/cup-wheel.htm   
6845   http://www.diamond-tool.net/product/diamond-bl...   
7742   http://www.diamond-tool.net/product/best-

In [15]:
cleaned_df.to_csv(r'B:\Python\Veridion Project\Project 4\cleaned_products.csv', index=False)